Le but de ce TP est de manipuler des CNN dans leur scénario principal d'utilisation : la Computer Vision. Il est à noter que cette architecture peut être utilisée en NLP, en particulier lorsqu'on travaille sur des textes relativement courts, mais le mécanisme de convolution est plutôt à l'état de l'art lorsqu'on cherche à réaliser une tâche sur des images.

Nous allons réaliser un Transfer Learning sur un modèle VGG, en procédant en deux grands temps :
1. Extraction de features des images afin d'en réaliser une segmentation (apprentissage non supervisé) puis de comparer les résultats de cette segmentation avec les classes d'origine. L'idée est plutôt d'informer la faisabilité d'un moteur de classification d'images tout en étudiant au passage la pertinence du choix du référentiel de classes.
2. Classification (apprentissage supervisé) des images, avec tout d'abord une approche simple sur notre strict ensemble d'images, puis une approche avec data augmentation. La data augmentation peut être réalisée grâce à un data generator (comme illustré dans les slides) - les images sont alors directement récupérées à la volée dans le repertoire des images - ou être intégrée au modèle (layer(s) en début de modèle).

Mettre à jour au fur et à mesure la cellule suivante des imports.

In [ ]:
from glob import glob
import os
from typing import Any, Protocol, Sequence

from dotenv import load_dotenv
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
class Predictor(Protocol):
    def predict(self, args: Any, **kwargs: Any) -> Any:
        ...

# TEMPS 1 : SEGMENTATION

## Préparation du dataset

<font color="purple">Récupérer le chemin du dossier contenant les images du fichier .env.</font>

In [ ]:
load_dotenv()
IMG_FOLDER = os.getenv("TP2_IMG_FOLDER")

<font color="purple">Initialiser une constante contenant la liste référentielle des labels possibles.</font>

In [ ]:
LABELS_LIST = ["cloudy", "rain", "shine", "sunrise"]

<font color="purple">En observation la convention de nommage des fichiers images, écrire une fonction (efficace) permettant de déterminer le label d'une photo à partir du nom du fichier.</font>

In [ ]:
def get_image_label(
    img_filename: str,
    possible_labels: Sequence[str]|None = None,
    label_not_found_value: str = "unknown",
) -> str:
    """Extract the label from the name of the image file."""
    if possible_labels is None:
        possible_labels = LABELS_LIST
    for label in possible_labels :
        if label in img_filename:
            return label
    return label_not_found_value

<font color="purple">Ecrire une fonction créant un dataframe pandas contenant le chemin vers l'image et le label. Construire le dataframe associé à notre jeu de données.</font>

In [ ]:
def create_data_from_img_files(path: str = IMG_FOLDER) -> pd.DataFrame:
    """Create a DataFrame containing all the paths to the images as well
    as the labels of these images.
    """
    images_list = glob(f"{path}*\*.jp*")  # in case some files are not jpg or jpeg files
    df_out = pd.DataFrame(data={"image_path": images_list})
    df_out["label_name"] = df_out["image_path"].apply(get_image_label)
    return df_out

img_df = create_data_from_img_files()

In [ ]:
img_df.head()

<font color="purple">Encoder le label à l'aide du LabelEncoder de scikit-learn.</font>

In [ ]:
le = LabelEncoder()
img_df["label"] = le.fit_transform(img_df["label_name"])

<font color="purple">Vérifier le nombre d'images dans chaque classe.</font>

In [ ]:
img_df.groupby("label").count()

<font color="purple">La cellule suivante (utilisant matplotlib) permet de visualiser 3 images côte à côte représentatives de chaque label. L'exécuter pour vérifier la cohérence de la sortie.</font>

In [ ]:
for label_name in LABELS_LIST:
    mask = img_df["label_name"]==label_name
    image_path_list = img_df[mask].reset_index().loc[0:2, "image_path"]
    for i in range(3):
        image = imread(image_path_list[i])  
        plt.subplot(131 + i)
        plt.imshow(image)
    plt.title(label_name)
    plt.show() 

## Feature extraction

### Création du modèle pré-entraîné

<font color="purple">Charger le modèle VGG16 de tensorflow.keras.applications.vgg16.</font> Pour plus de renseignements sur l'architecture de VGG16, il est possible de consulter par exemple la ressource suivante : https://datacorner.fr/vgg-transfer-learning/.

In [ ]:
base_model = VGG16()

<font color="purple">Afficher les 5 dernières couches du modèle à l'aide de .layers.</font>

In [ ]:
display(base_model.layers[-5:])

L'ouput n'est pas très lisible. <font color="purple">Afficher un résumé de l'architecture du modèle avec .summary().</font>

In [ ]:
base_model.summary()

<font color="purple">Construire un nouveau modèle à partir du modèle VGG16 amputé de toutes ses n-1 dernières couches Dense, où n désigne le nombre de couches Dense successives en aval du modèle.</font> Pour cela, utiliser instancier un objet de la classe Model du module tensorflow.keras.models, en passant comme arguments du constructeurs :
- inputs: les inputs du modèle VGG16,
- outputs: le résultat de l'appel de .output sur la liste des couches du modèle VGG16 amputé des n-1 dernières couches.

<font color="purple">Afficher un résumé de l'architecture du modèle.</font>

In [ ]:
# Votre code ici

## Création des features des images

<font color="purple">Préparer une fonction prenant en entrée le chemin vers une image et renvoyant les features extraites grâce au modèle, en appliquant les étapes successives suivantes :</font>
- chargement de l'image à l'aide de la fonction load_img de tensorflow.keras.preprocessing.image, en fixant une taille de 224x224,
- conversion en array à l'aide de la fonction img_to_array de tensorflow.keras.preprocessing.image également,
- ajout d'un axe à partir de l'axe 0 à l'aide de la fonction expand_dims de numpy (pour en visualiser l'utilité, essayer de commenter la ligne dans la fonction et constater l'erreur lors de l'appel sur une image),
- prétraitement pour normaliser le tableau représentant l'image avec la fonction standard preprocess_input de tensorflow.keras.applications.vgg16 (sans se poser plus de questions : les modèles keras viennent généralement avec leur fonction de prétraitement des images adaptées de manière spécifique au modèle en question),
- prédiction des features à partir de la méthode predict du modèle.

Il est à noter qu'il n'est pas classique d'avoir un predict dans une fonction "préparatoire". Ici nous nous le permettons car l'extraction de feature est préparatoire au X qui retendra dans le modèle de segmentation. Lorsque nous utiliserons notre CNN pour réaliser la classification complète, nous sortirons cet appel de la préparation des données.

In [ ]:
def extract_features(image_path: str, model: Predictor) -> np.array:
    """Extract features from an image given its path."""
    img_array = img_to_array(load_img(image_path, target_size=(224, 224)))
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return model.predict(img_array, verbose=False)[0]

<font color="purple">Appliquer cette fonction à l'ensemble du dataset ligne par ligne. Convertir le résultat en array numpy de taille nombre d'images x nombre de features à l'aide de la fonction np.stack (appliquer auparavant .values sur les features extraites).</font>

In [ ]:
# Votre code ici

<font color="purple">Afficher la taille de ce tableau.</font>

In [ ]:
# Votre code ici

## Réduction de dimension

### Réduction de dimension via ACP pour l'extraction même de features

<font color="purple">Appliquer une simple ACP pour réduire la dimension de l'image. Choisir par exemple de conserver un nombre de composantes tel que la quantité de variance expliquée soit supérieure à 99%. Vérifier la taille du tableau de features en sortie de l'ACP.</font>

In [ ]:
# Votre code ici

### Réduction de dimension T-SNE pour la visualisation

<font color="purple">Projeter les features en dimension 2 pour la visualisation, grâce à un TSNE. Consulter la documentation de ce modèle.</font>

In [ ]:
# Votre code ici

<font color="purple">Reformer un dataframe contenant l'ensemble des features post-TSNE ainsi qu'une dernière colonne contenant le label. En afficher un échantillon aléatoire de taille 10.</font>

In [ ]:
# Votre code ici

Voici une fonction (nous la réutiliserons plus tard) permettant d'afficher le nuage de points des individus sur l'espace de projection de dimension (cette fois en combinant plt avec un scatterplot seaborn, pour varier les plaisirs).

Même si nous n'utiliserons pas les vraies classes pour segmenter ce nuage de points, nous affichons avec des couleurs différentes les points ayant des labels différents à fin purement informative. Le nom de la colonne du dataframe à partir de laquelle colorer les points est un argument d'entrée de notre fonction d'affichage.

In [ ]:
def create_tsne_viz(tsne_df: pd.DataFrame, hue_col: str) -> None:
    plt.figure(figsize=(10,5))
    sns.scatterplot(
        tsne_df,
        x="tsne_one",
        y="tsne_two",
        hue=hue_col,
    )
    plt.title(f"TSNE et visualisation - {hue_col}.")
    plt.xlabel("Première composante")
    plt.ylabel("Seconde composante")
    plt.show()

<font color="purple">Appeler cette fonction sur notre jeu de données :</font>

In [ ]:
# Votre code ici

Quelques interprétations :
- L'analyse graphique donne des indices visuels quand à la faisabilité de la séparation automatique des images selon leurs vraies classes -> se faire une idée sur la question. Cela permet d'estimer la faisabilité de réaliser ultérieurement une classification supervisée pour déterminer automatiquement les classes des images.
- Cette étape est plutôt rapide à réaliser, notamment parce qu'elle ne demande l'entraînement de task nets. Une conclusion négative sur la faisabilité peut permettre d'éviter de réaliser des traitements plus lourds. Cela permet également de discuter du caractère "naturel" du découpage en labels déjà effectué (il serait peut-être possible de découper ce même dataset en 4 groupes plus isolés, sur la base de l'extraction de features effectuées).
- On peut image que cette démarche en plusieurs étapes s'inscrive dans la démarche agile d'un projet data.

### Clustering

Il s'agit ici de conforter simplement, par une mesure de l'ARI, l'analyse graphique précédente qui semble montrer la faisabilité relative d'une classification supervisée, avec des difficultés à prévoir pour segmenter les images "shine". Cette mesure de l'ARI nécessite de créer des clusters théoriques.
Pour réaliser une mesure de ce que nous voyons graphiquement, il faut partir des données en sorties du t-SNE. Pour toute autre application (classification ou segmentation réelle des images), il faudra repartir des features extraites par l'ensemble des étapes dédiées à cela, c'est-à-dire en sortie de l'ACP mais en amont du t-SNE.
Le score ARI est lié aux labels des données ; aussi rien ne garantit que la "meilleure" solution de feature engineering, si tant est qu'elle existe, sera celle qui génère le meilleur ARI.


Nous faisons ici le choix de mettre en oeuvre l'algorithme K-means. <font color="purple">Mettre en oeuvre un tel modèle en fixant un nombre de clusters égal au nombre de classes anticipé (4).</font> Il est possible de repartir de la sortie de l'ACP ou de celle du t-SNE (à discuter).

In [ ]:
# Votre code ici

<font color="purple">Ajouter au dataframe des données extraites pour l'ACP ou le t-SNE une colonne contenant le numéro du cluster affecté par K-means. En afficher un échantillon aléatoire de taille 10.</font>

In [ ]:
# Votre code ici

<font color="purple">Effectuer à l'oeil une correspondance (bijective) entre les classes et les clusters (il est bien entendu possible d'automatiser la recherche d'une telle correspondance maximisant la métrique de similarité, mais ici, cela peut être réalisé visuellement).</font>

In [ ]:
# Votre code ici

<font color="purple">Dénombrer le nombre d'images shine affectées au cluster cloudy.</font>

In [ ]:
# Votre code ici

<font color="purple">Afficher une image shine considérée comme cloudy par le clustering. Essayer d'interpréter d'où peut venir la confusion (si possible).</font>

In [ ]:
# Votre code ici

Selon les images obtenues, la confusion peut venir de la présence de neige interprétée comme des nuages, la présence d'un ciel nuageux sans pluie avec du soleil, la présence d'un autre élément remarquable mais non indicateur de la classe (par exemple l'océan), la présence de features "non robustes" telles qu'une colorimétrie très particulière de l'image... Il est aussi possible de tomber sur une image pour laquelle la cause de la confusion n'est pas apparente à l'oeil humain !

Il est désormais temps d'avoir une analyse un peu plus globale des éléments cohérents et de la confusion.

### Analyse par classes

<font color="purple">A l'aide de la fonction confusion_matrix du module metrics de scikit-learn, afficher la matrice de confusion brute de notre problème. Veiller à bien réaligner les classes et les clusters. Une fois la matrice brute obtenue, appeler la fonction de la cellule suivante pour améliorer le visuel.</font>

In [ ]:
# Votre code ici

In [ ]:
sns.heatmap(
    conf_mat,
    annot=True,
    fmt="g",
    cmap="Blues",
    xticklabels=LABELS_LIST,
    yticklabels=LABELS_LIST,
)
plt.xlabel("Cluster")
plt.ylabel("Classe réelle")
plt.show()

<font color="purple">Générer également un "rapport de classification" (toujours à partir du module metrics de scikit-learn).</font>

In [ ]:
# Votre code ici

<font color="purple">Pour terminer, calculer une mesure de similarité globale telle que le ARI score. Une fonction permettant de le calculer directement est (bien entendu) disponible dans scikit-learn. A votre avis, dans quel module ?</font> Rechercher sur Internet la fonction exacte à utiliser.

In [ ]:
# Votre code ici

<font color="purple">Cette fonction est-elle symétrique ? (Peut-on inverser valeurs réelles et prédites ?)</font>

In [ ]:
# Votre réponse ici

<font color="purple">Nous arrivons à la fin de la première partie. Avant de tout casser avec des bêtises, c'est peut-être le moment de sauvegarder les variables d'intérêt. Choisir les variables les plus importantes et les sauvegarder dans le dossier courant au format pickle.</font>

In [ ]:
# Votre code ici

# TEMPS 2 : CLASSIFICATION

<font color="orange">En repartant du code précédent pour cette étape, écrire le prétraitement que suivra cette image, en supprimant l'inférence par le modèle. </font> Pour rappel, les étapes suivantes seront les suivantes :
- charger l'image avec la fonction load_img du module tensorflow.keras.preprocessing.image, en utilisant l'argument target_size pour forcer une redimension fixée à 224x224,
- convertir l'image en array avec la fonction img_to_array de ce même module,
- appliquer la fonction de préprocessing déjà prête de VGG16 : preprocess_input du module tensorflow.keras.applications.vgg16.

In [ ]:
# Votre code ici

<font color="orange">Si ce n'est pas déjà fait, à partir du traitement unitaire écrit précédemment, écrire la fonction prenant en entrée notre dataframe et créant le tableau de toutes les images contenues dans le dataframe (dans l'ordre).</font>

In [ ]:
# Votre code ici

<font color="orange">Appeler cette fonction sur notre dataframe d'images.</font> Ce traitement peut prendre du temps.

In [ ]:
# Votre code ici

<font color="orange">Préparer le vecteur y en appelant to_categorical sur la colonne contenant le label (numérique).</font>

In [ ]:
# Votre code ici

<font color="orange">Séparer le jeu de données en un jeu d'entraînement et un jeu de test. Fixer un random_state pour garantir la reproductibilité des résultats pendant la phase d'élaboration de notre modèle. Stratifier selon la cible.</font>

In [ ]:
# Votre code ici

Remarque : généralement cette étape est effectuée AVANT tout autre traitement afin d'éviter tout data leakage. Pour autant, ici, la fonction de préprocessing s'applique aux images de façon unitaire (le traitement d'un image n'est pas dépendant des autres images), et, étant donné la stratification, toutes les classes du référentiel seront nécessairement représentées dans le jeu d'entraînement et dans celui de test, de sorte que l'encoding de la cible dans le jeu de test ne révèlera pas de surprise de type modalité inconnue.

<font color="orange">Redécouper le jeu d'entraînement en mini-jeu d'entraînement et jeu de validation.</font>

In [ ]:
# Votre code ici

### Préparation du modèle

<font color="orange">Ecrire une fonction permettant de créer notre modèle VGG16.</font> Pour cela, en s'inspirant du début de travail déjà réalisé dans la partie segmentation :
- instancier un objet VGG16. A partir de la doc, faire en sorte d'initialiser les points à ceux du modèle pré-entraîné sur le jeu ImageNet. Choisir par exemple de ne pas inclure les 3 premières couches fully-connected, et spécifier dans ce cas une taille d'entrée de 224x224x3
- indiquer que toutes ces couches seront "gelées" (non entraînables). Pour cela, boucler sur l'ensemble des éléments de model.layers et définir l'attribut trainable à False.
- stocker dans une variable model.output, puis ajouter une couche de pooling, une couche dense, une couche de dropout et une dernière couche dense.
- définir le nouveau modèle et le compiler.

In [ ]:
# Votre code ici

<font color="orange">Créer effectivement le modèle, avec un context manager spécifiant le device à utiliser (c'est ici qu'on spécifierait quel GPU utiliser si on en disposer). Dans notre cas, utilisons "/cpu:0"</font>

In [ ]:
# Votre code ici

<font color="orange">Créer un "callback" pour créer un early stopping de la manière suivante et sauvegarder le modèle de la manière suivante :</font>

In [ ]:
# Votre code ici

<font color="orange">A l'intérieur du context manager spécifiant le device à utiliser, lancer l'entraînement du modèle. Prévoir dès maintenant de sauvegarder la sortie dans une variable appelée history (nous l'utiliserons plus tard). Attention à bien préciser le jeu de validation ! </font>

In [ ]:
# Votre code ici

<font color="orange">Vérifier le score du dernier epoch sur le jeu d'entraînement et le jeu de validation grâce à la méthode evaluate de notre modèle entraîné.</font>

In [ ]:
# Votre code ici

<font color="orange">Vérifier le score de l'epoch optimal sur le jeu de validation et le jeu de test.</font>

In [ ]:
# Votre code ici

<font color="orange">Consulter la doc de plot_keras_history. A partir de celle-ci, visualiser l'évolution de la loss et de l'accuracy au fil des epochs.</font>

In [ ]:
# Votre code ici

<font color="orange">Afficher les classes réelles et prédites sur les images du jeu de validation.</font>

In [ ]:
# Votre code ici

<font color="orange">De manière analogue à la section précédente, afficher la matrice de confusion associée ainsi que le rapport de classification.</font>

In [ ]:
# Votre code ici

<font color="orange">Essayer éventuellement de visualiser les prédictions sur les images de test. <br> Il serait en particulier intéressant de voir la prédiction du modèle sur des images présentant des écueils soulevés dans la section précédente (par exemple des images avec de la neige) qui n'auraient pas été utilisées dans le jeu d'entraînement, s'il y en a.</font>

In [ ]:
# Votre code ici

### Data augmentation (intégrée au modèle)

Une ressource utile est celle de tensorflow sur la data augmentation, située à l'adresse suivante : https://www.tensorflow.org/tutorials/images/data_augmentation

Grâce au tutorial (première méthode), créer une data augmentation contenant RandomFlip, RandomRotation et RandomZoom.

In [ ]:
# Votre code ici

<font color="orange">Reprendre l'ensemble des questions précédentes de la partie CLASSIFICATION et comparer les deux modèles.</font>

In [ ]:
# Votre code ici